In [1]:
# Import necessary libraries
import h2o
from h2o.automl import H2OAutoML

# Initialize the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu120.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5i1x4b6m
  JVM stdout: /tmp/tmp5i1x4b6m/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5i1x4b6m/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_unknownUser_2ijby4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
# Load your data into a H2OFrame
data = h2o.import_file('/kaggle/input/optiver-trading-at-the-close/train.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
# Specify the response and predictor columns
response_column = 'target'
predictor_columns = data.columns
predictor_columns.remove("target")

In [4]:
# Need to detect best model on reduced dataset due to size
## Determine the ratio for your sample
ratio = 0.01
## Split frame
splits = data.split_frame(ratios = [ratio], seed = 1234)
# Sample is the first split
sample = splits[0]
print("sample size train+val is:",sample.nrows)

# Split the sample into training and validation sets
train_sample, val_sample = sample.split_frame(ratios=[0.8], seed=1)

# Initialize and train the H2OAutoML model
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(max_models=10,include_algos=["DeepLearning"])
aml.train(x=predictor_columns, y=response_column, training_frame=train_sample)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

# Predict on the validation set
preds = aml.predict(val_sample)

## Convert the predictions back to a pandas dataframe (optional)
#preds_df = h2o.as_list(preds)

sample size train+val is: 52388
AutoML progress: |
20:40:38.511: _train param, Dropping bad and constant columns: [row_id]

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                  rmse      mse      mae    rmsle    mean_residual_deviance
DeepLearning_grid_2_AutoML_1_20231011_204038_model_3   9.39066  88.1844  6.3869       nan                   88.1844
DeepLearning_grid_2_AutoML_1_20231011_204038_model_12  9.39456  88.2578  6.38095      nan                   88.2578
DeepLearning_grid_2_AutoML_1_20231011_204038_model_5   9.39862  88.3341  6.37879      nan                   88.3341
DeepLearning_grid_3_AutoML_1_20231011_204038_model_3   9.40954  88.5395  6.39923      nan                   88.5395
DeepLearning_grid_2_AutoML_1_20231011_204038_model_1   9.41718  88.6832  6.39445      nan                   88.6832
DeepLearning_grid_1_AutoML_1_20231011_204038_model_1   9.41992  88.7349  6.39644      nan              

In [5]:
# Extract best model
best_model = aml.leader

# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_grid_2_AutoML_1_20231011_204038_model_3'

In [6]:
# Split the data into training and validation sets
train, val = data.split_frame(ratios=[0.8], seed=1)

In [7]:
# Continue training the model on the full data set
best_model.train(x=predictor_columns, y=response_column, training_frame=train)

deeplearning Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: Dropping bad and constant columns: [row_id]
  warnings.warn(mesg["message"], RuntimeWarning)


█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1697056803730_7


Status of Neuron Layers: predicting target, regression, gaussian distribution, Quadratic loss, 43,601 weights/biases, 521.9 KB, 1,474,331 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  -----------------------
    1        15       Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.036611316557275134   0.09541717171669006    0.0         0.01927791476502049    0.19278424978256226  0.30244515687713946    0.11822527647018433
    3        200      Rectifier  0.0        0.0   0.0   0.09051021923764474    0.11003971099853516    0.0         -0.031927463540824055  0.07768410444259644  0.9252175438337633     0.02576175332069397
    4        1        Linear                0.0   0.0   0.0010525266815602664  0.0007116908673197031  0.0         -0.005439480587260732  0.04708237946033478  -0.022148869742030196  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 89.53980187378882
RMSE: 9.462547324784634
MAE: 6.313117736891289
RMSLE: NaN
Mean Residual Deviance: 89.53980187378882

Scoring History: 
    timestamp            duration          training_speed    epochs     iterations    samples      training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  ---------  ------------  -----------  ---------------  -------------------  --------------  -------------
    2023-10-11 21:52:12  0.000 sec                           0          0             0            nan              nan                  nan             nan
    2023-10-11 21:52:19  8.190 sec         6571 obs/sec      0.010649   1             44628        9.54612          91.1283              6.36956         0.00457484
    2023-10-11 21:52:28  16.729 sec        8927 obs/sec      0.031991   3             134068       9.50966          90.4337              6.35438         0.0121628
    2023-10-11 21:52:35  24.225 sec        10083 obs/sec     0.0533998  5             223788       9.50173          90.283               6.34206         0.013809
    2023-10-11 21:52:42  31.373 sec        10811 obs/sec     0.074818   7             313547       9.48802          90.0226              6.34818         0.0166533
    2023-10-11 21:52:49  38.127 sec        11346 obs/sec     0.0959827  9             402244       9.48095          89.8884              6.33195         0.0181188
    2023-10-11 21:52:56  45.433 sec        11606 obs/sec     0.117296   11            491563       9.47903          89.852               6.32956         0.018516
    2023-10-11 21:53:03  52.424 sec        11851 obs/sec     0.138691   13            581226       9.4956           90.1665              6.34471         0.0150814
    2023-10-11 21:53:10  59.218 sec        12073 obs/sec     0.159985   15            670466       9.46743          89.6322              6.30613         0.0209172
    2023-10-11 21:53:16  1 min  5.674 sec  12324 obs/sec     0.181345   17            759980       9.46978          89.6768              6.31032         0.0204304
    2023-10-11 21:53:23  1 min 12.379 sec  12478 obs/sec     0.202658   19            849297       9.46255          89.5398              6.31312         0.0219267
    2023-10-11 21:53:30  1 min 19.460 sec  12538 obs/sec     0.2239     21            938319       9.46567          89.5989              6.31141         0.021281
    2023-10-11 21:53:37  1 min 26.148 sec  12662 obs/sec     0.245155   23            1.0274e+06   9.48489          89.9631              6.33959         0.0173028
    2023-

In [8]:
# Save the model
h2o.save_model(model=best_model, path="/kaggle/working/", force=True)

'/kaggle/working/DeepLearning_model_python_1697056803730_7'

In [9]:
# Predict on the validation set
preds = best_model.predict(val)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
